## ⚙️ 환경설정

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
print('⚙️  torth 설정 완료')
from sklearn.metrics import average_precision_score
import numpy as np
from tqdm import tqdm
import collections
from torch.utils.data.sampler import Sampler
import matplotlib.pyplot as plt
from public_function import MPerClassSampler
print('⚙️  환경설정 완료')

⚙️  torth 설정 완료
⚙️  class정의 완료
⚙️  환경설정 완료


## 📊대조 학습용 데이터 로더

In [ ]:
transform = transforms.Compose([
    transforms.Resize((512, 512)),  # 이미지 크기 고정
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5071, 0.4867, 0.4408], std=[0.2675, 0.2565, 0.2761])
])

train_dataset = torchvision.datasets.ImageFolder(root='my_dataset', transform=transform)
sampler = MPerClassSampler(train_dataset.targets, m=4, length_before_new_iter=len(train_dataset))
batch_size = 44

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=False, sampler=sampler
)
print('⚙️  데이타셋 로드 및 로더 정의')

⚙️  데이타셋 로드 및 로더 정의


## ✨모델 로드 및 학습

In [ ]:
# 기본적인 모델
class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()
        resnet = torchvision.models.resnet18(pretrained=True)
        self.feature_extractor = nn.Sequential(*list(resnet.children())[:-1])

    def forward(self, x):
        x = self.feature_extractor(x)
        return x.view(x.size(0), -1)

#모델 GPU로 가능하면 옮기기
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNet18()
model = model.to(device)
print('⚙️  기본 모델 형태 로딩 완료')

c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


⚙️  기본 모델 형태 로딩 완료


In [5]:
# 빠른 손실값 계산 함수 정의
def contrastive_loss_fast(features, labels, margin=0.3):
    features = features / (torch.norm(features, p=2, dim=1, keepdim=True) + 1e-12).expand_as(features)
    distance_matrix = torch.cdist(features, features, p=2)
    labels_expanded = labels.unsqueeze(0).expand(len(labels), len(labels))
    label_matrix = (labels_expanded == labels_expanded.t()).byte()
    label_matrix.fill_diagonal_(0)  # Exclude self-comparison
    same_loss = label_matrix * distance_matrix.pow(2)
    different_loss = (1 - label_matrix) * torch.pow(torch.clamp(margin - distance_matrix, min=0.0), 2)
    loss = same_loss + different_loss
    return loss.mean()
print('⚙️  빠른 손실값 계산함수')

⚙️  빠른 손실값 계산함수


In [8]:
epoch = 30
margin = 0.75
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5, weight_decay=0.0001)
for i in range(epoch):
    progress_bar = tqdm(train_dataloader)  # 진행바를 출력하여, 학습 과정을 눈으로 보기 쉽게 만들어 줍니다!

    for data in progress_bar:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        features = model(images)
        loss = contrastive_loss_fast(features, labels, margin)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        progress_bar.set_postfix({"현재 Epoch": i, "현재 대조 손실 값": f"{loss.item():.4f}"})
print(f'⚙️  학습이 완료됨')

torch.save(model.state_dict(), "trained_model.pth")
print("💾 모델이 'trained_model.pth' 파일로 저장되었습니다!")

  0%|          | 0/50 [00:00<?, ?it/s]c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
 94%|█████████▍| 47/50 [11:52<00:46, 15.39s/it, 현재 Epoch=0, 현재 대조 손실 값=0.0353]c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\TiffImagePlugin.py:890: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 2. 
  warnings.warn(str(msg))
100%|██████████| 50/50 [12:30<00:00, 15.00s/it, 현재 Epoch=29, 현재 대조 손실 값=0.0155]

⚙️  학습이 완료됨
💾 모델이 'trained_model.pth' 파일로 저장되었습니다!
